In [78]:
import pandas as pd
import psycopg2

## Cargar la base de datos

In [79]:
try:
    conn = psycopg2.connect(
        host='localhost',
        user='postgres',
        password='root',
        database='arriendos',
        port='5432'
    )  # type: ignore
    print('Conexión exitosa')
    cur = conn.cursor()
    sql = '''select u.nombre, u.fecha_ingreso, u.precio_pactado, d.departamento, d.id_departamento, 
            p.mes_pago, p.year_pago, p.monto_pago, p.fecha_pago from usuarios u
            join departamentos d on d.id_usuario = u.id_usuario
            join Pagos p on u.id_usuario = p.id_usuario;'''
    cur.execute(sql)
    datos = cur.fetchall()
    columnas = [descripcion[0] for descripcion in cur.description]
    cur.close()
    conn.close()
except Exception as ex:
    print(f'Error: {ex}')
else:
    print('Cerrando sesión')

Conexión exitosa
Cerrando sesión


In [81]:
df = pd.DataFrame(datos, columns=columnas)


## Procesamiento datos

In [84]:
meses_numeros = {'ENE': 1, 'FEB': 2, 'MAR': 3, 'ABR': 4, 'MAY': 5, 'JUN': 6, 'JUL': 7, 'AGO': 8, 'SEP': 9, 'OCT': 10, 'NOV': 11, 'DIC': 12}

In [112]:
# Crear una nueva columna con los números de los meses para todos los registros
df['mes_numero'] = df['mes_pago'].map(meses_numeros)

In [111]:
# Crea un dataframe para ver el nombre de los usuarios y sus pagos segun los meses
pivot_df = df.pivot_table(index='nombre', columns='mes_numero', values=['monto_pago', 'fecha_pago'], aggfunc={'monto_pago': 'sum', 'fecha_pago': 'first'})
pivot_df.columns = pivot_df.columns.swaplevel()
pivot_df = pivot_df.sort_index(axis=1)


In [69]:
# Cambia los numeros por el mes
iniciales_meses = list(map(lambda x: (next(key for key, value in meses_numeros.items() if value == x[0]), x[1]), pivot_df.columns))
iniciales_meses = list(map(lambda x: (x[0],x[1].replace('fecha_pago', 'fecha')), iniciales_meses))
iniciales_meses = list(map(lambda x: (x[0],x[1].replace('monto_pago', 'pago')), iniciales_meses))
pivot_df.columns = pd.MultiIndex.from_tuples(iniciales_meses)

In [89]:
pivot_df[[('ENE', 'fecha')]]

,ENE
,fecha
nombre,
Anita Chicaiza,2024-01-10
Emma Simbaña,2024-01-08
Fernando Pichucho,2024-01-22
Jeremy mestanza,2024-01-23
Martha Lucia Barona,2024-02-19
Nestor Castro,2024-02-29


In [110]:
pivot_df.iloc[:,pivot_df.columns.get_level_values(1) == 'fecha']

,ENE,FEB,MAR,ABR
,fecha,fecha,fecha,fecha
nombre,,,,
Anita Chicaiza,2024-01-10,2024-02-14,2024-03-11,2024-04-07
Emma Simbaña,2024-01-08,2024-02-07,2024-03-08,2024-04-08
Fernando Pichucho,2024-01-22,2024-03-15,2024-03-30,NaN
Jeremy mestanza,2024-01-23,2024-02-26,2024-03-28,NaN
Martha Lucia Barona,2024-02-19,2024-02-26,2024-03-15,2024-04-06
Nestor Castro,2024-02-29,2024-02-29,NaN,NaN
